In [7]:
import os
import pandas as pd 
import shutil
import configparser

In [8]:
def getBandsIndexes(cat,bands):
    return [list(cat.columns).index(c)+1 for c in bands]

In [9]:
def makeConfig(dirCat:str,fileNameCat:str,idcolumn:str,zspeccolumn:str,
               features:list,expNum:int,dirConfigs:str,configName:str):
    
    cat=pd.read_csv(os.path.join(dirCat,fileNameCat))
    headers=list(cat)
    idcolumnidx=str(int(headers.index('Seq')+1))
    zspeccolumnidx=str(int(headers.index('photoZ_SED')+1))
    featuresIdx=getBandsIndexes(cat,features)
    featuresIdxStr=','.join(str(x) for x in featuresIdx)
    nNeuronsLayer1=str(int(2*len(featuresIdx)+1))
    nNeuronsLayer2=str(int(len(featuresIdx)-1))

    newIni='exp'+str(expNum).zfill(3)+'_'+configName+'.ini'
    shutil.copyfile(os.path.join(dirConfigs,fileConfigExample),os.path.join(dirConfigs,newIni))
    parser=configparser.RawConfigParser()
    parser.read(os.path.join(dirConfigs,newIni))
    parser.set('Train Input Info','photoColumnIdxList',featuresIdxStr)
    parser.set('Train Input Info','dirPath',dirCat)
    parser.set('Train Input Info','idcolumnidx',idcolumnidx)
    parser.set('Train Input Info','zspeccolumnidx',zspeccolumnidx)
    parser.set('Train Input Info','name',fileNameCat)
    parser.set('Test Input Info','photoColumnIdxList',featuresIdxStr)
    parser.set('Test Input Info','name',fileNameCat)
    parser.set('Test Input Info','dirPath',dirCat)
    parser.set('Test Input Info','idcolumnidx',idcolumnidx)
    parser.set('Test Input Info','zspeccolumnidx',zspeccolumnidx)
    parser.set('Output Info','dirpath','Sources\COSMOS_photoZ\COSMOS_v3.0\experimentsArticle\exp'+str(expNum).zfill(3))
    parser.set('Output Info','modelSubDirName','ex'+str(expNum).zfill(3))
    parser.set('Learning Info','nHiddenNeurons',nNeuronsLayer1+','+nNeuronsLayer2)
    parser.set('Train Output Info','nameSuffix','_'+fileNameCat)
    parser.set('Test Output Info','nameSuffix','_'+fileNameCat)
    parser.set('Log Info','nameSuffix','_'+fileNameCat)
    parser.set('Restart Info','nameSuffix','_'+fileNameCat)

    with open(os.path.join(dirConfigs,newIni), 'w') as configfile:
        parser.write(configfile)
    return

In [10]:
dirCat=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\catalogs\forPaper\v06'
fileList = [f for f in os.listdir(dirCat) if os.path.isfile(os.path.join(dirCat, f))]
catNames=[f for f in fileList if '06_CleanCompare_' in f]
dirConfigs=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\cleanCompare'
fileConfigExample=r'exXX_phz_z_ConfigExample.ini'

In [11]:
catNames

['06_CleanCompare_noClean_12252.csv',
 '06_CleanCompare_OccupMap_12252.csv',
 '06_CleanCompare_SEDoutl_12252.csv',
 '06_CleanCompare_SpecZoutl_12252.csv']

In [12]:
bandsPhiLab=['IA527', 'IA624','IA679', 'IB574']
bandsPhiLab=[c+'ap3' for c in bandsPhiLab]
cat=pd.read_csv(os.path.join(dirCat,catNames[0]))
bandsBroad=[c for c in cat.columns if ('magap3' in c) and ('sc_' not in c)]
bandsNarrow=[c for c in cat.columns if ('ap3' in c) and ('sc_' not in c) and ('mag' not in c)]
bandsBroadNarrow=bandsBroad+bandsNarrow
bandsBroadPhiLab=bandsBroad+bandsPhiLab
expNum=45
fsets={'broad':bandsBroad,'broadNarrow':bandsBroadNarrow,'broadPhiLab':bandsBroadPhiLab}

In [15]:
bandsBroadPhiLab

['Ksmagap3',
 'Ymagap3',
 'Hmagap3',
 'Jmagap3',
 'Bmagap3',
 'Vmagap3',
 'ipmagap3',
 'rmagap3',
 'umagap3',
 'zppmagap3',
 'IA527ap3',
 'IA624ap3',
 'IA679ap3',
 'IB574ap3']

In [16]:
for fileNameCat in catNames:
    for fsetName,fset in fsets.items():
        makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn='photoZ_SED',
               features=fset,expNum=expNum,dirConfigs=dirConfigs,
                   configName=fileNameCat.replace('06_','').replace('.csv','')+'_'+fsetName+'_SED')
        expNum+=1 
for fileNameCat in catNames:
    for fsetName,fset in fsets.items():
        makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn='specZ',
               features=fset,expNum=expNum,dirConfigs=dirConfigs,
                   configName=fileNameCat.replace('06_','').replace('.csv','')+'_'+fsetName+'_specZ')
        expNum+=1 

In [ ]:
os.chdir('D:\\')

# 05.11.2020. Checking out how different cleaning influences NMAD
configs=os.listdir(dirConfigs)
configs=[conf for conf in configs if 'CleanCompare_' in conf]
for conf in configs:
    conf=os.path.join(dirConfigs,conf)
    print(conf)
    os.system('metaphor_pipeline '+conf)
    print('Ready')

D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\cleanCompare\exp045_CleanCompare_noClean_12252_broad_SED.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\cleanCompare\exp046_CleanCompare_noClean_12252_broadNarrow_SED.ini
